In [1]:
import numpy as np
from IPython.display import clear_output, display
from pydrake.all import (AbstractValue,
                         AddMultibodyPlantSceneGraph,
                         DiagramBuilder,
                         JointSliders,
                         LeafSystem,
                         MeshcatVisualizer, 
                         Parser,
                         RigidTransform,
                         RollPitchYaw, 
                         StartMeshcat)

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad, AddPackagePaths


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [7]:
class PrintPose(LeafSystem): 
    
    def __init__(self, body_index): 
        LeafSystem.__init__(self)
        self.__body_index__ = body_index
        self.DeclareAbstractInputPort(
            'body_poses', 
            AbstractValue.Make([RigidTransform()])
        )
        self.DeclareForcedPublishEvent(self.Publish)
    
    def Publish(self, context): 
        pose = self.get_input_port().Eval(context)[self.__body_index__]
        print(pose)
        print('gripper at (m): ' + np.array2string(pose.translation(),
                                                   formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print('gripper RPY (Rad): ' + 
              np.array2string( RollPitchYaw(pose.rotation()).vector(),
                              formatter={'float': lambda x: "{:3.2f}".format(x)}
                             ))
        clear_output(wait=True) # from IPython display
        # clear_output clean all the out for each invoke, aka i'll see only the last print
        
def gripper_forward_kinematics_example(): 
    builder = DiagramBuilder()
    
    # build the system 
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    AddPackagePaths(parser)
    parser.AddAllModelsFromFile(
        FindResource("models/iiwa_and_wsg.dmd.yaml")
    )
    plant.Finalize()
    
    # DRAW THE FRAME
    BODY_NAMES = ["iiwa_link_1", "iiwa_link_2",
                  "iiwa_link_3", "iiwa_link_4",
                  "iiwa_link_5", "iiwa_link_6", 
                  "iiwa_link_7", "body"]

    for body_name in BODY_NAMES: 
        # add the frame of each join 
        AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)
        
    
    meshcat.Delete() # IN THEORY DELETE ALL OBJ OF A GIVEN PATH
    meshcat.DeleteAddedControls() # delete all button 
    
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, 
        scene_graph.get_query_output_port(),
        meshcat
    )
    
    
    wsg = plant.GetModelInstanceByName('wsg')
    gripper = plant.GetBodyByName('body', wsg) # CHECK WHY DOUBLE PARAMS
    print_pose = builder.AddSystem(PrintPose(gripper.index()))
    builder.Connect(
        plant.get_body_poses_output_port(), 
        print_pose.get_input_port()
    )
    
    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()
    
    

In [ ]:
gripper_forward_kinematics_example()

RigidTransform(
  R=RotationMatrix([
    [-0.186042292953882, -0.8395724786209475, 0.5103981958968323],
    [0.6143659388964242, 0.3059895294746856, 0.727272233057042],
    [-0.7667742551470923, 0.4488746606728201, 0.45887774041620627],
  ]),
  p=[-0.12308542904804168, 0.4270051897757752, 0.9451889551135606],
)
gripper at (m): [-0.12 0.43 0.95]
gripper RPY (Rad): [0.77 0.87 1.86]
